In [ ]:
%tensorflow_version 2.x

In [ ]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [ ]:
path=get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8')as f:
  text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893
total chars: 57


In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285
Vectorization...


In [ ]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=30,
          callbacks=[print_callback])

Epoch 1/30
200285/200285 [==============================] - 221s 1ms/step - loss: 1.8646

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "icians. it was different with felix mend"
icians. it was different with felix mendous and string the strong of the morality of the superion of the strong its spect of the properation of the spect and instruction of the stronger and the strong of the strong the some the would in the strenger and strong the some the strong the stronger and strong and a stronger and strenges the strong it is the superion of the properation of the strong the precises of the stronger and and and the
----- diversity: 0.5
----- Generating with seed: "icians. it was different with felix mend"
icians. it was different with felix mender to streng the proche of the most of every one belowfulle to the screed and from the also and from the precises is the spirits its greem of the power and
first of which has the stronger and precised and pro

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 about well vains isalot of all
that follyly of pride, firstly, magy is those he again eyees the dringn)
in phallbk will behindned and lead himself point of rejuicts," and the insisitely namely, in concert herderabve, securies to "shee with at "b
Epoch 19/30
200285/200285 [==============================] - 223s 1ms/step - loss: 1.3294

----- Generating text after Epoch: 18
----- diversity: 0.2
----- Generating with seed: "hen revenge, as with the ancient
greeks,"
hen revenge, as with the ancient
greeks, the man is the sense of the superiors of the conception of the probably species of the superiors of the spirituality of the conditional such a strength, the conditional man who seems to the strength, and superficial of the probably superstition, the ordinary interpretent, the man in the conditional conditional sense and and all the problem of the transible to the superiors of the probably superio
----- diversity: 0.5
----- Generating with seed: "hen revenge, as with the ancient
greeks,"